**⚠️ Original Code in error️**

In [1]:
%matplotlib inline

In [2]:
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Dropout, Flatten, MaxPooling2D

In [3]:
class CNN(Model):
    def __init__(model, nb_classes, in_shape=None):
        model.nb_classes = nb_classes
        model.in_shape = in_shape
        model.build_model()
        super().__init__(model.x, model.y)
        model.compile()

    def build_model(model):
        nb_classes = model.nb_classes
        in_shape = model.in_shape

        x = Input(in_shape)

        h = Conv2D(32, kernel_size=(3, 3), activation='relu',
                   input_shape=in_shape)(x)
        h = Conv2D(64, (3, 3), activation='relu')(h)
        h = MaxPooling2D(pool_size=(2, 2))(h)
        h = Dropout(0.25)(h)
        h = Flatten()(h)
        z_cl = h

        h = Dense(128, activation='relu')(h)
        h = Dropout(0.5)(h)
        z_fl = h

        y = Dense(nb_classes, activation='softmax', name='preds')(h)

        model.cl_part = Model(x, z_cl)
        model.fl_part = Model(x, z_fl)

        model.x, model.y = x, y

    def compile(model):
        Model.compile(model, loss='categorical_crossentropy',
                      optimizer='adadelta', metrics=['accuracy'])

In [4]:
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [5]:
class Dataset:
    def __init__(self, X, y, nb_classes, scaling=True, test_size=0.2, random_state=0):
        self.X = X
        self.add_channels()

        X = self.X
        X_train, X_test, y_train, y_test =\
            train_test_split(X, y, test_size=test_size, random_state=random_state)

        print(X_train.shape, y_train.shape)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')

        if scaling:
            scaler = MinMaxScaler()
            n = X_train.shape[0]
            X_train = scaler.fit_transform(X_train.reshape(n, -1)).reshape(X_train.shape)
            n = X_test.shape[0]
            X_test = scaler.transform(X_test.reshape(n, -1)).reshape(X_test.shape)

            self.scaler = scaler

        print('X_train shape: ', X_train.shape)
        print(X_train.shape[0], ' train samples')
        print(X_test.shape[0], ' test samples')

        Y_train = np_utils.to_categorical(y_train, nb_classes)
        Y_test = np_utils.to_categorical(y_test, nb_classes)

        self.X_train, self.X_test = X_train, X_test
        self.y_train, self.y_test = y_train, y_test
        self.Y_train, self.Y_test = Y_train, Y_test

    def add_channels(self):
        X = self.X

        if len(X.shape) == 3:
            N, img_rows, img_cols = X.shape

            if K.image_dim_ordering() == 'th':
                X = X.reshape(X.shape[0], 1, img_rows, img_cols)
                input_shape = (1, img_rows, img_cols)
            else:
                X = X.reshape(X.shape[0], img_rows, img_cols, 1)
                input_shape = (img_rows, img_cols, 1)
        else:
            input_shape = X.shape[1:]

        self.X = X
        self.input_shape = input_shape

In [6]:
from datetime import datetime
import os
import uuid

import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics

In [7]:
class Machine:
    def __init__(self, X, y, nb_classes=2, fig=True):
        self.nb_classes = nb_classes
        self.set_data(X, y)
        self.set_model()
        self.fig = fig

    def set_data(self, X, y):
        nb_classes = self.nb_classes
        self.data = Dataset(X, y, nb_classes)
        print('data.input_shape', self.data.input_shape)

    def set_model(self):
        nb_classes = self.nb_classes
        data = self.data
        self.model = CNN(nb_classes, data.input_shape)

    def fit(self, epochs=10, batch_size=128, verbose=1):
        data = self.data
        model = self.model

        history = model.fit(
            data.X_train, data.Y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=verbose,
            validation_data=(data.X_test, data.Y_test)
        )
        return history

    def run(self, epochs=100, batch_size=128, verbose=1):
        data = self.data
        model = self.model
        fig = self.fig

        history = self.fit(epochs=epochs, batch_size=batch_size, verbose=verbose)
        score = model.evaluate(data.X_test, data.Y_test, verbose=0)

        print('Confusion Matrix')
        Y_test_pred = model.predict(data.X_test, verbose=0)
        y_test_pred = np.argmax(Y_test_pred, axis=1)
        print(metrics.confusion_matrix(data.y_test, y_test_pred))

        print('Test Score:', score[0])
        print('Test Accuracy:', score[1])

        suffix = self.unique_filename('datetime')
        foldname = 'output_' + suffix
        os.makedirs(foldname)
        self.save_history_history('history_history.npz', history.history, foldname)
        model.save_weights(os.path.join(foldname, 'dl_model.h5'))
        print('Output results are saved in', foldname)

        if fig:
            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)
            self.plot_acc(history)
            plt.subplot(1, 2, 2)
            self.plot_loss(history)

            plt.show()

        self.history = history

        return foldname



    @staticmethod
    def unique_filename(type='uuid'):
        if type == 'datetime':
            filename = datetime.now().strftime('%y%m%d_%H%M%S')
        elif type == 'uuid':
            filename = str(uuid.uuid4())

        return filename

    @staticmethod
    def save_history_history(fname, history_history, fold=''):
        np.save(os.path.join(fold, fname), history_history)

    @staticmethod
    def plot_acc(history):
        plt.plot(history.history['acc'])
        plt.plot(history.history['val_acc'])
        plt.title('Model Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc=0)

    @staticmethod
    def plot_acc(history):
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc=0)

In [8]:
from keras import datasets

In [9]:
class Cifar10Machine(Machine):
    def __init__(self):
        (X, y), (x_test, y_test) = datasets.cifar10.load_data()
        super(Cifar10Machine, self).__init__(X, y, nb_classes=10)

In [10]:
def main():
    m = Cifar10Machine()
    m.run()

In [11]:
main()

(40000, 32, 32, 3) (40000, 1)
X_train shape:  (40000, 32, 32, 3)
40000  train samples
10000  test samples
data.input_shape (32, 32, 3)


RuntimeError: It looks like you are subclassing `Model` and you forgot to call `super(YourClass, self).__init__()`. Always start with this line.

**☕️ My Refactored Code**

In [7]:
%matplotlib inline

In [8]:
from keras import layers, models

In [21]:
class CNNModel(models.Model):
    def __init__(self, input_shape, num_classes):
        super(CNNModel, self).__init__()

        self.conv1 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape)
        self.conv2 = layers.Conv2D(32, kernel_size=(3, 3), activation='relu')
        self.pool = layers.MaxPooling2D(pool_size=(2, 2))
        self.dropout1 = layers.Dropout(0.25)
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(128, activation='relu')
        self.dropout2 = layers.Dropout(0.5)
        self.dense2 = layers.Dense(num_classes, activation='softmax')


    def call(self, x):
        net = self.conv1(x)
        net = self.conv2(net)
        net = self.pool(net)
        net = self.dropout1(net)
        net = self.flatten(net)
        net = self.dense1(net)
        net = self.dropout2(net)
        y = self.dense2(net)

        return y

    def get_config(self):
        base_config = super(CNNModel, self).get_config()
        return base_config

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils

In [23]:
class Dataset:
    def __init__(self, x, y, num_classes, scaling=True, test_size=0.2, random_state=2020):
        self.x = x
        self.y = y

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=random_state
        )

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')

        x_train_scaled = None
        x_test_scaled = None
        if scaling:
            min_max_scale = MinMaxScaler()

            num_train_sample = x_train.shape[0]
            image_width = x_train.shape[1]
            image_height = x_train.shape[2]
            image_channel = x_train.shape[3]

            x_train_scaled = min_max_scale\
                .fit_transform(x_train.reshape(num_train_sample, -1))\
                .reshape((num_train_sample, image_width, image_height, image_channel))

            num_test_sample = x_test.shape[0]

            x_test_scaled = min_max_scale\
                .fit_transform(x_test.reshape(num_test_sample, -1))\
                .reshape((num_test_sample, image_width, image_height, image_channel))

            self.input_shape = (image_width, image_height, image_channel)
            self.scale = min_max_scale

        y_train_encoded = np_utils.to_categorical(y_train, num_classes)
        y_test_encoded = np_utils.to_categorical(y_test, num_classes)

        self.x_train = x_train_scaled if scaling else x_train
        self.x_test = x_test_scaled if scaling else x_test
        self.y_train = y_train_encoded
        self.y_test = y_test_encoded

In [27]:
class Classifier:
    def __init__(self, x, y, num_classes, fig=True):
        self.num_classes = num_classes
        self.fig = fig

        self.data = Dataset(x, y, num_classes=num_classes)
        self.model = CNNModel(input_shape=self.data.input_shape, num_classes=num_classes)
        self.model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['acc'])

    def fit(self):
        x_train = self.data.x_train
        y_train = self.data.y_train

        self.model.fit(x_train, y_train, epochs=10, batch_size=128, verbose=1)

    def score(self):
        x_test = self.data.x_test
        y_test = self.data.y_test

        score = self.model.evaluate(x_test, y_test)
        print(score)

In [28]:
from keras import datasets

In [29]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
classifier = Classifier(x_train, y_train, 10)
classifier.fit()
classifier.score()

Epoch 1/10
250/313 [======================>.......] - ETA: 4s - loss: 1.7311 - acc: 0.3681 

KeyboardInterrupt: 